In [1]:
import ee
import wxee
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta
import os

/Users/dhruvyadav/anaconda3/envs/NWS/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ee.Initialize()

In [3]:
region_polygon = ee.Geometry.Rectangle([-99, 29, -97, 31])

In [4]:
surface_variables = [
    "temperature_2m_above_ground",
    "specific_humidity_2m_above_ground",
    "relative_humidity_2m_above_ground",
    "u_component_of_wind_10m_above_ground",
    "v_component_of_wind_10m_above_ground",
    "precipitable_water_entire_atmosphere"
]

In [5]:
start_date = datetime(2025, 2, 1)
end_date   = datetime(2025, 2, 8)

In [6]:
all_datasets = []
current = start_date

In [7]:
out_dir = "gfs_2025_1_week_data"
os.makedirs(out_dir, exist_ok=True)

In [ ]:
while current < end_date:
    next_day = current + timedelta(days=1)
    
    current_str = current.strftime("%Y-%m-%d")
    next_day_str = next_day.strftime("%Y-%m-%d")
    
    gfs_subset = (
        ee.ImageCollection("NOAA/GFS0P25")
        .filterDate(current_str, next_day_str)
        .filterBounds(region_polygon)
        .select(surface_variables)
        .filterMetadata("forecast_hours", "less_than", 24) 
    )
    
    # Number of Images
    count = gfs_subset.size().getInfo()
    print(f"Date: {current_str}, Images found: {count}")
    
    # If count=0, skip or handle error
    if count == 0:
        print("No data for this day, skipping.")
        current = next_day
        continue
    
    # Convert subset to xarray
    try:
        xr_day = gfs_subset.wx.to_xarray(
            region=region_polygon,
            scale=25000,   # ~25 km
            progress=True,
            num_cores=1,
            masked=True
        )
    except Exception as e:
        print(f"Error downloading data for {current_str}: {e}")
        current = next_day
        continue
    
    # Inspect dataset
    print(xr_day)  # dims, coords, variables
    # e.g., print(xr_day["temperature_2m_above_ground"])
    
    # Save locally as NetCDF
    out_file = os.path.join(out_dir, f"gfs_{current_str}.nc")
    xr_day.to_netcdf(out_file)
    print(f"Saved {out_file}")
    
    # Move to the next day
    current = next_day

Date: 2025-02-01, Images found: 96


Requesting data:   0%|          | 0/96 [03:35<?, ?it/s]
/Users/dhruvyadav/anaconda3/envs/NWS/lib/python3.10/site-packages/wxee/utils.py:149: UserWarning: Different non-null values were encountered for the same variable at the same time coordinate. The first value was taken.
  warnings.warn(


<xarray.Dataset> Size: 19kB
Dimensions:                               (time: 4, x: 10, y: 10)
Coordinates:
  * time                                  (time) datetime64[ns] 32B 2025-02-0...
  * x                                     (x) float64 80B -98.93 ... -96.91
  * y                                     (y) float64 80B 31.1 30.88 ... 29.08
    spatial_ref                           int64 8B 0
Data variables:
    temperature_2m_above_ground           (time, y, x) float64 3kB 14.94 ... nan
    specific_humidity_2m_above_ground     (time, y, x) float64 3kB 0.003361 ....
    relative_humidity_2m_above_ground     (time, y, x) float64 3kB 30.5 ... nan
    u_component_of_wind_10m_above_ground  (time, y, x) float64 3kB 1.31 ... nan
    v_component_of_wind_10m_above_ground  (time, y, x) float64 3kB -3.253 ......
    precipitable_water_entire_atmosphere  (time, y, x) float64 3kB 4.346 ... nan
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  1

Requesting data:   0%|          | 0/96 [02:51<?, ?it/s]
/Users/dhruvyadav/anaconda3/envs/NWS/lib/python3.10/site-packages/wxee/utils.py:149: UserWarning: Different non-null values were encountered for the same variable at the same time coordinate. The first value was taken.
  warnings.warn(


<xarray.Dataset> Size: 19kB
Dimensions:                               (time: 4, x: 10, y: 10)
Coordinates:
  * time                                  (time) datetime64[ns] 32B 2025-02-0...
  * x                                     (x) float64 80B -98.93 ... -96.91
  * y                                     (y) float64 80B 31.1 30.88 ... 29.08
    spatial_ref                           int64 8B 0
Data variables:
    temperature_2m_above_ground           (time, y, x) float64 3kB 17.01 ... nan
    specific_humidity_2m_above_ground     (time, y, x) float64 3kB 0.003962 ....
    relative_humidity_2m_above_ground     (time, y, x) float64 3kB 31.3 ... nan
    u_component_of_wind_10m_above_ground  (time, y, x) float64 3kB 0.2188 ......
    v_component_of_wind_10m_above_ground  (time, y, x) float64 3kB 4.39 ... nan
    precipitable_water_entire_atmosphere  (time, y, x) float64 3kB 5.081 ... nan
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  1

...

Requesting data:   0%|          | 0/96 [03:40<?, ?it/s]
/Users/dhruvyadav/anaconda3/envs/NWS/lib/python3.10/site-packages/wxee/utils.py:149: UserWarning: Different non-null values were encountered for the same variable at the same time coordinate. The first value was taken.
  warnings.warn(


<xarray.Dataset> Size: 19kB
Dimensions:                               (time: 4, x: 10, y: 10)
Coordinates:
  * time                                  (time) datetime64[ns] 32B 2025-02-0...
  * x                                     (x) float64 80B -98.93 ... -96.91
  * y                                     (y) float64 80B 31.1 30.88 ... 29.08
    spatial_ref                           int64 8B 0
Data variables:
    temperature_2m_above_ground           (time, y, x) float64 3kB 21.12 ... nan
    specific_humidity_2m_above_ground     (time, y, x) float64 3kB 0.01245 .....
    relative_humidity_2m_above_ground     (time, y, x) float64 3kB 75.7 ... nan
    u_component_of_wind_10m_above_ground  (time, y, x) float64 3kB 2.83 ... nan
    v_component_of_wind_10m_above_ground  (time, y, x) float64 3kB 6.942 ... nan
    precipitable_water_entire_atmosphere  (time, y, x) float64 3kB 23.5 ... nan
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  1 

Requesting data:   0%|          | 0/96 [03:20<?, ?it/s]


<xarray.Dataset> Size: 19kB
Dimensions:                               (time: 4, x: 10, y: 10)
Coordinates:
  * time                                  (time) datetime64[ns] 32B 2025-02-0...
  * x                                     (x) float64 80B -98.93 ... -96.91
  * y                                     (y) float64 80B 31.1 30.88 ... 29.08
    spatial_ref                           int64 8B 0
Data variables:
    temperature_2m_above_ground           (time, y, x) float64 3kB 25.8 ... nan
    specific_humidity_2m_above_ground     (time, y, x) float64 3kB 0.009653 ....
    relative_humidity_2m_above_ground     (time, y, x) float64 3kB 44.2 ... nan
    u_component_of_wind_10m_above_ground  (time, y, x) float64 3kB -0.677 ......
    v_component_of_wind_10m_above_ground  (time, y, x) float64 3kB 4.105 ... nan
    precipitable_water_entire_atmosphere  (time, y, x) float64 3kB 18.18 ... nan
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  1

/Users/dhruvyadav/anaconda3/envs/NWS/lib/python3.10/site-packages/wxee/utils.py:149: UserWarning: Different non-null values were encountered for the same variable at the same time coordinate. The first value was taken.
  warnings.warn(


In [6]:
start_date = datetime(2025, 1, 10)
end_date   = datetime(2025, 1, 18)

In [7]:
all_datasets = []
current = start_date

In [8]:
out_dir = "gfs_2025_other_week_data"
os.makedirs(out_dir, exist_ok=True)

In [9]:
while current < end_date:
    next_day = current + timedelta(days=1)
    
    current_str = current.strftime("%Y-%m-%d")
    next_day_str = next_day.strftime("%Y-%m-%d")
    
    gfs_subset = (
        ee.ImageCollection("NOAA/GFS0P25")
        .filterDate(current_str, next_day_str)
        .filterBounds(region_polygon)
        .select(surface_variables)
        .filterMetadata("forecast_hours", "less_than", 24) 
    )
    
    # Number of Images
    count = gfs_subset.size().getInfo()
    print(f"Date: {current_str}, Images found: {count}")
    
    # If count=0, skip or handle error
    if count == 0:
        print("No data for this day, skipping.")
        current = next_day
        continue
    
    # Convert subset to xarray
    try:
        xr_day = gfs_subset.wx.to_xarray(
            region=region_polygon,
            scale=25000,   # ~25 km
            progress=True,
            num_cores=1,
            masked=True
        )
    except Exception as e:
        print(f"Error downloading data for {current_str}: {e}")
        current = next_day
        continue
    
    # Inspect dataset
    print(xr_day)  
    
    # Save locally as NetCDF
    out_file = os.path.join(out_dir, f"gfs_{current_str}.nc")
    xr_day.to_netcdf(out_file)
    print(f"Saved {out_file}")
    
    # Move to the next day
    current = next_day

Date: 2025-01-10, Images found: 96


Requesting data:   0%|          | 0/96 [04:05<?, ?it/s]
/Users/dhruvyadav/anaconda3/envs/NWS/lib/python3.10/site-packages/wxee/utils.py:149: UserWarning: Different non-null values were encountered for the same variable at the same time coordinate. The first value was taken.
  warnings.warn(


<xarray.Dataset> Size: 19kB
Dimensions:                               (time: 4, x: 10, y: 10)
Coordinates:
  * time                                  (time) datetime64[ns] 32B 2025-01-1...
  * x                                     (x) float64 80B -98.93 ... -96.91
  * y                                     (y) float64 80B 31.1 30.88 ... 29.08
    spatial_ref                           int64 8B 0
Data variables:
    temperature_2m_above_ground           (time, y, x) float64 3kB 0.4526 ......
    specific_humidity_2m_above_ground     (time, y, x) float64 3kB 0.004125 ....
    relative_humidity_2m_above_ground     (time, y, x) float64 3kB 100.0 ... nan
    u_component_of_wind_10m_above_ground  (time, y, x) float64 3kB 1.329 ... nan
    v_component_of_wind_10m_above_ground  (time, y, x) float64 3kB -0.8721 .....
    precipitable_water_entire_atmosphere  (time, y, x) float64 3kB 22.2 ... nan
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  

Requesting data:   0%|          | 0/96 [02:52<?, ?it/s]
/Users/dhruvyadav/anaconda3/envs/NWS/lib/python3.10/site-packages/wxee/utils.py:149: UserWarning: Different non-null values were encountered for the same variable at the same time coordinate. The first value was taken.
  warnings.warn(


<xarray.Dataset> Size: 19kB
Dimensions:                               (time: 4, x: 10, y: 10)
Coordinates:
  * time                                  (time) datetime64[ns] 32B 2025-01-1...
  * x                                     (x) float64 80B -98.93 ... -96.91
  * y                                     (y) float64 80B 31.1 30.88 ... 29.08
    spatial_ref                           int64 8B 0
Data variables:
    temperature_2m_above_ground           (time, y, x) float64 3kB 3.838 ... nan
    specific_humidity_2m_above_ground     (time, y, x) float64 3kB 0.002425 ....
    relative_humidity_2m_above_ground     (time, y, x) float64 3kB 46.5 ... nan
    u_component_of_wind_10m_above_ground  (time, y, x) float64 3kB 2.105 ... nan
    v_component_of_wind_10m_above_ground  (time, y, x) float64 3kB -4.968 ......
    precipitable_water_entire_atmosphere  (time, y, x) float64 3kB 5.121 ... nan
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  

...

Requesting data:   0%|          | 0/96 [02:46<?, ?it/s]
/Users/dhruvyadav/anaconda3/envs/NWS/lib/python3.10/site-packages/wxee/utils.py:149: UserWarning: Different non-null values were encountered for the same variable at the same time coordinate. The first value was taken.
  warnings.warn(


<xarray.Dataset> Size: 19kB
Dimensions:                               (time: 4, x: 10, y: 10)
Coordinates:
  * time                                  (time) datetime64[ns] 32B 2025-01-1...
  * x                                     (x) float64 80B -98.93 ... -96.91
  * y                                     (y) float64 80B 31.1 30.88 ... 29.08
    spatial_ref                           int64 8B 0
Data variables:
    temperature_2m_above_ground           (time, y, x) float64 3kB 10.18 ... nan
    specific_humidity_2m_above_ground     (time, y, x) float64 3kB 0.00348 .....
    relative_humidity_2m_above_ground     (time, y, x) float64 3kB 43.5 ... nan
    u_component_of_wind_10m_above_ground  (time, y, x) float64 3kB -1.054 ......
    v_component_of_wind_10m_above_ground  (time, y, x) float64 3kB -2.52 ... nan
    precipitable_water_entire_atmosphere  (time, y, x) float64 3kB 8.168 ... nan
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  

Requesting data:   0%|          | 0/96 [02:22<?, ?it/s]


<xarray.Dataset> Size: 19kB
Dimensions:                               (time: 4, x: 10, y: 10)
Coordinates:
  * time                                  (time) datetime64[ns] 32B 2025-01-1...
  * x                                     (x) float64 80B -98.93 ... -96.91
  * y                                     (y) float64 80B 31.1 30.88 ... 29.08
    spatial_ref                           int64 8B 0
Data variables:
    temperature_2m_above_ground           (time, y, x) float64 3kB 10.63 ... nan
    specific_humidity_2m_above_ground     (time, y, x) float64 3kB 0.002888 ....
    relative_humidity_2m_above_ground     (time, y, x) float64 3kB 34.8 ... nan
    u_component_of_wind_10m_above_ground  (time, y, x) float64 3kB -0.6637 .....
    v_component_of_wind_10m_above_ground  (time, y, x) float64 3kB 1.874 ... nan
    precipitable_water_entire_atmosphere  (time, y, x) float64 3kB 11.29 ... nan
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  

/Users/dhruvyadav/anaconda3/envs/NWS/lib/python3.10/site-packages/wxee/utils.py:149: UserWarning: Different non-null values were encountered for the same variable at the same time coordinate. The first value was taken.
  warnings.warn(


In [8]:
from datetime import datetime, timedelta

# Set your specific day (for instance, February 1, 2025)
current_date = datetime(2025, 2, 1)
next_date = current_date + timedelta(days=1)
current_str = current_date.strftime("%Y-%m-%d")
next_day_str = next_date.strftime("%Y-%m-%d")

In [9]:
# Test for forecast hour 6 on that day
gfs_forecast6 = (
    ee.ImageCollection("NOAA/GFS0P25")
    .filterDate(current_str, next_day_str)
    .filterBounds(region_polygon)
    .filterMetadata("forecast_hours", "equals", 6)
    .select(surface_variables)
)

print("Testing for:", current_str)
print("Forecast 6 Images Count:", gfs_forecast6.size().getInfo())

Testing for: 2025-02-01
Forecast 6 Images Count: 4


In [ ]:
gfs_forecast6.